In [1]:
import pickle
import os
from od.social import SocialGroup
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import BS_PRESET,SUMO_SIM_SECONDS

In [2]:
interest_config = InterestConfig(ResourceAllocatorType.NOMA_OPT,True,1.6,14)
statistic = None
print(interest_config)
with open("data/yQoS/{}/{}/object.pickle".format(interest_config.rng_seed,str(interest_config)), "rb") as file:
    statistic = pickle.load(file)

res_alloc_type(NOMA_OPT) req_rsu(True) traffic_scale(1.6)


In [3]:
class GV:
    NET_STATION_CONTROLLER = [
            bs_config for bs_config in BS_PRESET.values()
             if ((bs_config["type"] == BaseStationType.UMA)  or 
            (interest_config.req_rsu and bs_config["type"] == BaseStationType.UMI))
    ]
class STATISTIC:
    def Log(data):
        print(data)

In [4]:
GV.NET_STATION_CONTROLLER

[{'color': (0, 0, 0, 255),
  'pos': (160, 100),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (450, 330),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (127, 217),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (52, 145),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (315, 395),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (270, 50),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 

In [5]:
print(len(statistic.sg_header[SocialGroup.RCWS].keys()))
print(len(statistic.sg_header[SocialGroup.CRASH].keys()))

32707
9566


In [6]:

for sg in SocialGroup:
    ot_count = 0 
    for header, record in statistic.sg_header[sg].items():
        if record.is_src_ot > 0:
            ot_count +=1
    print(ot_count)
    print(ot_count / len(statistic.sg_header[sg]))
            

0
0.0
0
0.0


In [7]:

class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len = round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [8]:
target_header = "flow_2.53-269"
print("C2G?:{}".format(target_header in statistic.sg_header[SocialGroup.RCWS] and target_header in statistic.sg_header[SocialGroup.CRASH]))

C2G?:False


In [9]:
statistic.sg_header[SocialGroup.RCWS][target_header].time_bs_txq

[[[187.649, 187.802], [187.803, 187.803]], [], [], [], [], [], []]

In [10]:
statistic.sg_header[SocialGroup.RCWS][target_header].time_bs_tx

[[[187.802, 187.803], [187.803, 187.804]], [], [], [], [], [], []]

In [11]:
statistic.sg_header[SocialGroup.RCWS][target_header].at

187.647

In [12]:
statistic.sg_header[SocialGroup.RCWS][target_header].time_bs_serv

[187.803, -1, -1, -1, -1, -1, -1]

In [13]:
bs_index = 0

In [14]:
bs_serv_list = []
for header, record in statistic.sg_header[SocialGroup.RCWS].items():
    if(record.time_bs_serv[bs_index] > 0):
        bs_serv_list.append(Helper(header,record,bs_index))
bs_serv_list.sort(key=lambda x: (x.time_gen*1000)*10000000 + (x.time_len*1000) )

In [15]:
bs_serv_list_crit = []
for header, record in statistic.sg_header[SocialGroup.CRASH].items():
    if(record.time_bs_serv[bs_index] > 0):
        bs_serv_list_crit.append(Helper(header,record,bs_index))
bs_serv_list_crit.sort(key=lambda x: (x.time_gen*1000)*10000000 + (x.time_len*1000))

In [16]:
for helper in bs_serv_list_crit:
    print(str(helper))

flow_0.82-44[ at:186.001 ,to:186.002, dur:0.001, size:6704]
flow_0.69-82[ at:186.014 ,to:186.015, dur:0.001, size:6952]
flow_9.7-82[ at:186.016 ,to:186.017, dur:0.001, size:5824]
flow_0.60-82[ at:186.017 ,to:186.018, dur:0.001, size:3448]
flow_0.62-83[ at:186.017 ,to:186.018, dur:0.001, size:7896]
flow_0.77-82[ at:186.017 ,to:186.018, dur:0.001, size:3672]
flow_13.2-83[ at:186.017 ,to:186.018, dur:0.001, size:4232]
flow_2.65-82[ at:186.017 ,to:186.018, dur:0.001, size:6216]
flow_9.6-83[ at:186.017 ,to:186.018, dur:0.001, size:8584]
flow_0.55-82[ at:186.018 ,to:186.019, dur:0.001, size:6344]
flow_0.61-83[ at:186.018 ,to:186.019, dur:0.001, size:3856]
flow_0.66-83[ at:186.018 ,to:186.019, dur:0.001, size:4880]
flow_0.67-83[ at:186.018 ,to:186.019, dur:0.001, size:2632]
flow_0.76-83[ at:186.018 ,to:186.019, dur:0.001, size:3848]
flow_0.79-82[ at:186.018 ,to:186.019, dur:0.001, size:2880]
flow_2.62-82[ at:186.018 ,to:186.019, dur:0.001, size:8032]
flow_2.63-81[ at:186.018 ,to:186.019, dur:

In [17]:
for helper in bs_serv_list:
    print(str(helper))

flow_0.67-81[ at:186.0 ,to:186.017, dur:0.017, size:8848]
flow_2.49-81[ at:186.0 ,to:186.017, dur:0.017, size:11976]
flow_9.7-80[ at:186.0 ,to:186.017, dur:0.017, size:2736]
flow_2.44-81[ at:186.0 ,to:186.018, dur:0.018, size:13064]
flow_1.11-81[ at:186.001 ,to:186.018, dur:0.017, size:15992]
flow_1.7-81[ at:186.001 ,to:186.018, dur:0.017, size:7040]
flow_2.52-81[ at:186.002 ,to:186.018, dur:0.016, size:4800]
flow_0.43-81[ at:186.002 ,to:186.019, dur:0.017, size:5088]
flow_0.76-81[ at:186.002 ,to:186.019, dur:0.017, size:9520]
flow_0.80-81[ at:186.002 ,to:186.019, dur:0.017, size:7688]
flow_0.82-45[ at:186.002 ,to:186.019, dur:0.017, size:5720]
flow_0.48-81[ at:186.002 ,to:186.02, dur:0.018, size:12512]
flow_0.66-81[ at:186.002 ,to:186.02, dur:0.018, size:10456]
flow_1.8-81[ at:186.002 ,to:186.02, dur:0.018, size:12144]
flow_2.50-81[ at:186.002 ,to:186.021, dur:0.019, size:14456]
flow_0.54-81[ at:186.003 ,to:186.021, dur:0.018, size:6232]
flow_0.56-80[ at:186.003 ,to:186.021, dur:0.018

In [18]:
time_gap = {}
for helper in bs_serv_list_crit:
    at = str(helper.time_gen)
    to = str(helper.time_serv)
    if(to not in  time_gap):
        time_gap[to] = {"need": 0, "give": 0}
    if(at not in time_gap):
        time_gap[at] = {"need": 0, "give": 0}
    time_gap[to]["give"]+=helper.bits
    time_gap[at]["need"]+=helper.bits
keys = list(time_gap.keys())
keys.sort()
for key in keys:
    print("{}:{}".format(key,str(time_gap[key])))

186.001:{'need': 6704, 'give': 0}
186.002:{'need': 0, 'give': 6704}
186.014:{'need': 6952, 'give': 0}
186.015:{'need': 0, 'give': 6952}
186.016:{'need': 5824, 'give': 0}
186.017:{'need': 34048, 'give': 5824}
186.018:{'need': 46312, 'give': 34048}
186.019:{'need': 37424, 'give': 46312}
186.02:{'need': 26408, 'give': 37424}
186.021:{'need': 35408, 'give': 26408}
186.022:{'need': 27832, 'give': 35408}
186.023:{'need': 27040, 'give': 27832}
186.024:{'need': 4160, 'give': 27040}
186.025:{'need': 0, 'give': 4160}
186.027:{'need': 5464, 'give': 0}
186.028:{'need': 0, 'give': 5464}
186.035:{'need': 3120, 'give': 0}
186.036:{'need': 0, 'give': 3120}
186.048:{'need': 5112, 'give': 0}
186.049:{'need': 2576, 'give': 5112}
186.05:{'need': 45384, 'give': 2576}
186.051:{'need': 27400, 'give': 45384}
186.052:{'need': 43440, 'give': 27400}
186.053:{'need': 26208, 'give': 43440}
186.054:{'need': 48904, 'give': 26208}
186.055:{'need': 30232, 'give': 48904}
186.056:{'need': 24552, 'give': 30232}
186.057:{

In [19]:
time_gap = {}
for helper in bs_serv_list:
    at = str(helper.time_gen)
    to = str(helper.time_serv)
    if(to not in  time_gap):
        time_gap[to] = {"need": 0, "give": 0}
    if(at not in time_gap):
        time_gap[at] = {"need": 0, "give": 0}
    time_gap[to]["give"]+=helper.bits
    time_gap[at]["need"]+=helper.bits
keys = list(time_gap.keys())
keys.sort()
for key in keys:
    print("{}:{}".format(key,str(time_gap[key])))

186.0:{'need': 36624, 'give': 0}
186.001:{'need': 23032, 'give': 0}
186.002:{'need': 82384, 'give': 0}
186.003:{'need': 110776, 'give': 0}
186.004:{'need': 101696, 'give': 0}
186.005:{'need': 103304, 'give': 0}
186.006:{'need': 125536, 'give': 0}
186.007:{'need': 77432, 'give': 0}
186.008:{'need': 85040, 'give': 0}
186.009:{'need': 27488, 'give': 0}
186.01:{'need': 23040, 'give': 0}
186.011:{'need': 50552, 'give': 0}
186.012:{'need': 37928, 'give': 0}
186.013:{'need': 37232, 'give': 0}
186.014:{'need': 94312, 'give': 0}
186.015:{'need': 74256, 'give': 0}
186.016:{'need': 100448, 'give': 0}
186.017:{'need': 85048, 'give': 23560}
186.018:{'need': 123160, 'give': 40896}
186.019:{'need': 81600, 'give': 28016}
186.02:{'need': 57992, 'give': 35112}
186.021:{'need': 77992, 'give': 41944}
186.022:{'need': 38832, 'give': 33792}
186.023:{'need': 35096, 'give': 34968}
186.024:{'need': 42776, 'give': 56328}
186.025:{'need': 28848, 'give': 60712}
186.026:{'need': 100248, 'give': 62184}
186.027:{'ne

In [20]:
list(time_gap.keys()).sort()